# Tutorial 3: Clipper in Action with Pong

We have already explored some of the features of Clipper (insert recap here). Now let's take a look at Clipper in action with Pong! Released by Atari in 1972, Pong was the first commercially successful video game. You can read more about it <a href="https://en.wikipedia.org/wiki/Pong">here</a>

The goal of this tutorial is to use Clipper to deploy several AI policies to play against, and in doing so explore:
1. Deploy models trained in your choice of framework to Clipper with a few lines of code by using an existing model container or writing your own
2. Easily update or roll back models in running applications
3. Run each model in a separate Docker container for simple cluster management and resource allocation

This tutorial will be broken up into 3 main parts:
#### 1. Starting Clipper and Deploying First Policy
#### 2. Training a Better Model
#### 3. Deploying an updated, trained model

## Part 1: Starting Clipper and Deploying First Policy

Install packaged env from requirements.txt (not sure if we should do this before opening jupyter notebook or not)

In [ ]:
#Import dependencies
from clipper_admin import ClipperConnection, DockerContainerManager
from clipper_admin.deployers import python as py_deployer
import random
import numpy as np
import pandas as pd
from sklearn import linear_model
import requests

this_ip = requests.get('http://ip.42.pl/raw').text
this_ip

Our first step is to get Clipper started and deploy our first AI policy: one that guesses which direction to move the paddle randomly. First, make sure Docker is running.

The cell below will start Clipper. You can run docker ps in your bash to see the Clipper Docker containers

In [ ]:
#Starting Clipper, make sure you have Docker running before you run this cell
clipper_conn = ClipperConnection(DockerContainerManager())
clipper_conn.stop_all()
clipper_conn.start_clipper()

Running the cell below will register an application in Clipper called "pong" and create a Clipper endpoint for our random policy at http://localhost:1337/pong/predict

In [ ]:
def random_predict(xs):
    '''Deploy a policy that returns a random choice from 0, 1, or 2.
    Remember that Clipper requires the output of the predict function to be a list of string objects'''
    action = random.randint(..., ...)
    return [...(action) for _ in xs]

py_deployer.create_endpoint(clipper_conn, name="pong", input_type="doubles", func=random_predict,
                            default_output="0", slo_micros=100000)

```python
def random_predict(xs):
    action = random.randint(0, 2)
    return [str(action) for _ in xs]

py_deployer.create_endpoint(clipper_conn, name="pong", input_type="doubles", func=random_predict,
                            default_output="0", slo_micros=100000)
```

Now that we have a model deployed, let's see how it works! Run the cell below to start the game

In [ ]:
from subprocess import Popen, PIPE

server_proc = Popen([
    "python", 
    "pong-server/pong-server.py", 
    "localhost:1337", 
    this_ip,
    "pong_server.log"], stdout=PIPE)

Congratulations! We have depolyed our first model to Clipper! Let's take a look at it by clicking the link below and pressing 1 to start the game.





In [ ]:
print(f"""
This is your link to pong game:
http://{this_ip}:4000/pong/
""")

In [ ]:
# server_proc.terminate()

## Part 2: Training a better model

As you probably noticed, the random-guessing policy did not perform well at all. In order to train a better model, we are going to use imitation learning. Let's play the game again, this time with 2 actual players, and we will collect data on how you play the game to train our model! 

Go to pong link we had above and press 2. Find someone next to you and play pong against them!!

<b>Part to collect data into a output.csv --> currently doing this in really hacky way to just get it to work, but trying to figure out better solution</b>

Now that we have our data, lets train a new model! First, run the cell below to clean the data and format it for scikit-learn's LogisticRegression model.

In [ ]:
df_data = pd.read_csv('out.csv')
df_data.columns = ["label","paddle_y","ball_x","ball_y","ball_dx","ball_dy","x_prev","y_prev"]

def convert_label(label):
    """Convert labels into numeric values"""
    if(label=="down"):
        return 1
    elif(label=="up"):
        return 2
    else:
        return 0

df_data['label'] = df_data['label'].apply(convert_label)
df_data.loc[:, "paddle_y":"y_prev"] = df_data.loc[:, "paddle_y":"y_prev"]/500.0

df_data.head()

In [ ]:
df_data.size

We are going to use the data to train a scikit-learn Logistic Regression model. You can read more about the particular model <a url="http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html">here</a>

In [ ]:
labels = df_data['label']
training_data= df_data.drop(['label'], axis=1)

model = linear_model.LogisticRegression()
model.fit(training_data, labels)



## Part 3: Deploying updated model

Now that we have an updated model, we can deploy that model onto Clipper. Once the system realizes there is a new version of the model, it will automatically switch to serving the newer version of the model.

![update_model](imgs/update_model.png)

In [ ]:
def predict(inputs):
    # model.predict returns a list of predictions
    preds = model.predict(inputs)
    return [str(p) for p in preds]

'''
TO-DO:
Fill in the line of code below to deploy a version 2 of the pong model container. 
It takes in inputs of type double and uses the predict function defined above.
'''
py_deployer.deploy_python_closure(clipper_conn, name="...", version=..., input_type="...", func=..., pkgs_to_install=["numpy","scipy", "pandas", "sklearn"])


```python
def predict(inputs):
    # model.predict returns a list of predictions
    preds = model.predict(inputs)
    return [str(p) for p in preds]

'''
TO-DO:
Fill in the line of code below to deploy a version 2 of the pong model container. 
It takes in inputs of type double and uses the predict function defined above.
'''
py_deployer.deploy_python_closure(clipper_conn, name="pong", version=2, input_type="doubles", func=predict, pkgs_to_install=["numpy","scipy", "pandas", "sklearn"])
```

You may notice that it seems like we only deployed the predict function to Clipper. However, Clipper will track the dependencies that the model in the predict function uses and include them in the model Docker container as dependencies. You can take a look at the logs above to see the Docker container downloading said dependencies.

Go to pong link we showed above , click 1, and notice how the game is now serving the updated model!

In [ ]:
# server_proc.terminate()

## 4. Conclusion

Just as a recap, here's what we did today:

1. Deployed an initial random policy to Clipper and served predictions
2. Trained a new model with data that you generated while playing pong
3. Deployed new version of model and seamlessly switch the model being served

By doing so, we've explored the following Clipper features:

1. Deploy models trained in your choice of framework to Clipper with a few lines of code by using an existing model container or writing your own
2. Easily update or roll back models in running applications
3. Run each model in a separate Docker container for simple cluster management and resource allocation


In [ ]:
!python stop_c.py
